In [1]:
!pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [18]:
from langchain_community.tools.tavily_search import TavilySearchResults
import os
os.environ['TAVILY_API_KEY']="xxx"

print(os.environ['TAVILY_API_KEY'])
# print(os.environ['TAVILY_API_KEY'])
search =  TavilySearchResults(max_results = 2, TAVILY_API_KEY=os.environ['TAVILY_API_KEY'])
search_results = search.invoke("what is the weather in SF")
print(search_results)

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
[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1724522387, 'localtime': '2024-08-24 10:59'}, 'current': {'last_updated_epoch': 1724521500, 'last_updated': '2024-08-24 10:45', 'temp_c': 20.6, 'temp_f': 69.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 230, 'wind_dir': 'SW', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 73, 'cloud': 75, 'feelslike_c': 20.6, 'feelslike_f': 69.1, 'windchill_c': 16.9, 'windchill_f': 62.5, 'heatindex_c': 17.0, 'heatindex_f': 62.5, 'dewpoint_c': 13.3, 'dewpoint_f': 55.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 9.5, 'gust_kph': 15.3}}"}, {'url': 'h

In [19]:
!pip install -qU langchain-openai


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4o")
response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [22]:
tools = [search]
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [23]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_8TKuFHkmd7XPNyIRQihYN5Hb', 'type': 'tool_call'}]


In [24]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='ef98c024-362a-44f4-876b-bada8b6abe11'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-9ff022f6-075e-4893-bfb1-016c19019cb2-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91})]

In [25]:
response = agent_executor.invoke({"messages": [HumanMessage(content="whats the weather in SF?")]})
response["messages"]

[HumanMessage(content='whats the weather in SF?', id='9c232a8f-1f16-4f6a-8d69-ee0a0b451ac1'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NluHidjAV3KHG8jA74gURTGZ', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6dd89f7b-8576-4071-95e2-c6facc86c19f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_NluHidjAV3KHG8jA74gURTGZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 22, 'total_tokens': 108}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \

In [26]:
for chunk in agent_executor.stream(
     {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("-----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nxfRgRdifa3GqWaSiIgoTPbw', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-736dcc21-773a-489a-8b14-74bd337492ce-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_nxfRgRdifa3GqWaSiIgoTPbw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 22, 'total_tokens': 108})]}}
-----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United Sta

In [27]:
async for event in agent_executor.astream_events({"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"):
    kind = event["event"]
    if kind == "on_chain_start":
        if event["name"] == "Agent": # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(f"Starting agent: {event['name']} with input: {event['data'].get('input')}")
    elif kind == "on_chain_end": # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
        if event["name"] == "Agent": 
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")
            

/Users/fangwen/classProject/DamoonGitHubSentinel/.venv/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1724523556, \'localtime\': \'2024-08-24 11:19\'}, \'current\': {\'last_updated_epoch\': 1724523300, \'last_updated\': \'2024-08-24 11:15\', \'temp_c\': 21.1, \'temp_f\': 70.0, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 8.1, \'wind_kph\': 13.0, \'wind_degree\': 310, \'wind_dir\': \'NW\', \'pressure_mb\': 1019.0, \'pressure_in\': 30.09, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 71, \'cloud\': 75, \'feelslike_c\': 21.1, \'feelslike_f\': 70.0, \'windc

In [28]:
# 加入记忆
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream({"messages": [HumanMessage(content="hi im bob!")]}, config):
    print(chunk)
    print("------")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-bf262800-872e-420b-906e-a65dce3d0ce4-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94})]}}
------


In [29]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 106, 'total_tokens': 121}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-c440bd7a-f80c-419c-8fe3-a55a68f35033-0', usage_metadata={'input_tokens': 106, 'output_tokens': 15, 'total_tokens': 121})]}}
----


In [30]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't have access to personal information like your name. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 84, 'total_tokens': 103}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-f895d6f5-64c7-4143-85b7-047f26a25134-0', usage_metadata={'input_tokens': 84, 'output_tokens': 19, 'total_tokens': 103})]}}
----
